**Dự báo xu hướng giá CK bằng thuật toán Rừng ngẫu nhiên**

1. Tải dữ liệu
2. Tạo biến Y=0 nếu giá CK giảm và biến Y=1 nếu giá CK tăng.
3. Sử dụng hàm RandomForestClassifier từ thư viện sklearn.
4. Đánh giá độ chính xác của mô hình thông qua các chỉ số Accuracy, Precision, recall, F1-score.

**VCB**

In [3]:
import pandas as pd
# Đọc dữ liệu từ file CSV (có thể thay bằng file của bạn)
file_path = r'D:\DeTaiNam2024\SolieuHong\VCB.csv'
df = pd.read_csv(file_path)
# Kiểm tra tiêu đề các cột
df.head()


,Unnamed: 0,time,open,high,low,close,volume
0,0,2015-01-05,14.03,14.21,13.90,14.12,310010
1,1,2015-01-06,14.12,15.05,14.12,15.05,1684600
2,2,2015-01-07,15.09,15.31,14.65,14.87,1430340
3,3,2015-01-08,14.92,15.49,14.87,15.31,904240
4,4,2015-01-09,15.45,16.38,15.45,16.33,2286680


In [ ]:
# Sắp xếp theo ngày
df = df.sort_values('time')

# Tạo cột Y: nếu giá đóng cửa hôm nay thấp hơn hôm qua, Y=0, nếu ngược lại Y=1
Y = (df['close'] > df['open']).astype(int)

# Hiển thị dữ liệu với cột Y
print(df[['time', 'close', 'Y']])

In [5]:
# Lấy đặc trưng (features) và biến mục tiêu (target)
X = df[['open', 'high', 'low', 'volume']]  # Các cột đặc trưng
y_db= df['close']  # Biến mục tiêu cho hồi quy

In [6]:
from sklearn.model_selection import train_test_split # tải hàm train_test_split
# Chia dữ liệu thành tập huấn luyện và tập kiểm tra
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)


In [7]:

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
import time
# Bắt đầu tính thời gian huấn luyện
start_time = time.time()


# Khởi tạo mô hình Rừng ngẫu nhiên cho phân loại
rf = RandomForestClassifier(n_estimators=100, random_state=42)

# Huấn luyện mô hình
rf.fit(X_train, Y_train)
# Kết thúc tính thời gian huấn luyện
train_time_cls = time.time() - start_time
print(f'Thời gian huấn luyện mô hình phân loại: {train_time_cls:.4f} giây')

# Bắt đầu tính thời gian dự đoán
start_time = time.time()

# Dự đoán trên tập kiểm tra
Y_pred = rf.predict(X_test)
# Kết thúc tính thời gian dự đoán
predict_time_cls = time.time() - start_time
print(f'Thời gian dự đoán mô hình phân loại: {predict_time_cls:.4f} giây')
# Đánh giá mô hình phân loại bằng Accuracy
accuracy = accuracy_score(Y_test,Y_pred)
print(f'Accuracy (Phân loại): {accuracy}')

# Báo cáo chi tiết kết quả dự đoán
print(classification_report(Y_test, Y_pred))


Thời gian huấn luyện mô hình phân loại: 1.8810 giây
Thời gian dự đoán mô hình phân loại: 0.0226 giây
Accuracy (Phân loại): 0.654320987654321
              precision    recall  f1-score   support

           0       0.66      0.73      0.69       262
           1       0.64      0.57      0.60       224

    accuracy                           0.65       486
   macro avg       0.65      0.65      0.65       486
weighted avg       0.65      0.65      0.65       486



**Chạy tất cả chứng khoán trong folder**

In [1]:
import os
import pandas as pd
import time
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.metrics import mean_squared_error, accuracy_score, classification_report

In [2]:
def process_stock_file(file_path):
    # Đọc dữ liệu từ file csv
    df = pd.read_csv(file_path)
    
    # Nếu không đủ dữ liệu, bỏ qua file
    if df.shape[1] < 5:
        return None

    # Tạo các đặc trưng (features) và biến mục tiêu
    X = df[['open', 'high', 'low', 'volume']]
    Y = (df['close'] > df['open']).astype(int)  # Phân loại: xu hướng tăng (1) hoặc giảm (0)

    # Chia dữ liệu cho hồi quy và phân loại
    X_train, X_test,Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)
    
    # Biến lưu kết quả
    results = {}
    results['file'] = os.path.basename(file_path)

    # Huấn luyện mô hình Rừng ngẫu nhiên cho hồi quy
    start_time = time.time()
    rf = RandomForestClassifier(n_estimators=100, random_state=42)
    rf.fit(X_train, Y_train)
    train_time = time.time() - start_time

    #Dự báo
    start_time = time.time()
    Y_pred = rf.predict(X_test)
    predict_time = time.time() - start_time

     # Tính độ chính xác
    accuracy = accuracy_score(Y_test, Y_pred)
    results['accuracy'] = accuracy
    results['train_time'] = train_time
    results['predict_time'] = predict_time

    # In kết quả chi tiết cho phân loại
    class_report = classification_report(Y_test, Y_pred, output_dict=True)
    results.update({
        'precision_0': class_report['0']['precision'],
        'recall_0': class_report['0']['recall'],
        'f1_score_0': class_report['0']['f1-score'],
        'precision_1': class_report['1']['precision'],
        'recall_1': class_report['1']['recall'],
        'f1_score_1': class_report['1']['f1-score'],
    })

    return results


In [3]:
def all_stocks(folder, output):
    # Danh sách lưu trữ kết quả cho mỗi file
    all_results = []

    # Duyệt qua từng file trong thư mục
    for file_name in os.listdir(folder):
        if file_name.endswith('.csv'):
            file_path = os.path.join(folder, file_name)
            print(f'Đang xử lý {file_name}...')
            result = process_stock_file(file_path)
            if result:
                all_results.append(result)

    # Chuyển danh sách kết quả thành DataFrame
    results_df = pd.DataFrame(all_results)

    # Ghi kết quả ra file CSV
    results_df.to_csv(output, index=False)
    print(f'Kết quả đã được ghi ra {output}')


In [5]:
# Đường dẫn đến thư mục chứa các file chứng khoán
folder = r'D:\DeTaiNam2024\SoLieu\CK_HNX_VNSTOCK'

# Đường dẫn đến file kết quả output
output = r'D:\DeTaiNam2024\SoLieu\CK_HNX_VNSTOCK\output\RandomForest_Classiffication_stock_results_HNX.csv'

# Chạy xử lý tất cả các file trong thư mục và lưu kết quả
all_stocks(folder, output)

Đang xử lý AAV.csv...
Đang xử lý ADC.csv...
Đang xử lý ALT.csv...
Đang xử lý AMC.csv...
Đang xử lý AME.csv...
Đang xử lý AMV.csv...
Đang xử lý API.csv...
Đang xử lý APS.csv...
Đang xử lý ARM.csv...
Đang xử lý ATS.csv...
Đang xử lý BAB.csv...
Đang xử lý BAX.csv...
Đang xử lý BBS.csv...
Đang xử lý BCC.csv...
Đang xử lý BCF.csv...
Đang xử lý BDB.csv...


C:\Users\Admin\AppData\Roaming\Python\Python39\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Admin\AppData\Roaming\Python\Python39\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Admin\AppData\Roaming\Python\Python39\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Đang xử lý BED.csv...
Đang xử lý BKC.csv...
Đang xử lý BNA.csv...
Đang xử lý BPC.csv...
Đang xử lý BSC.csv...
Đang xử lý BST.csv...
Đang xử lý BTS.csv...
Đang xử lý BTW.csv...
Đang xử lý BVS.csv...
Đang xử lý BXH.csv...
Đang xử lý C69.csv...
Đang xử lý CAG.csv...
Đang xử lý CAN.csv...
Đang xử lý CAP.csv...
Đang xử lý CCR.csv...
Đang xử lý CDN.csv...
Đang xử lý CEO.csv...
Đang xử lý CET.csv...
Đang xử lý CIA.csv...
Đang xử lý CJC.csv...
Đang xử lý CKV.csv...
Đang xử lý CLH.csv...
Đang xử lý CLM.csv...
Đang xử lý CMC.csv...
Đang xử lý CMS.csv...
Đang xử lý CPC.csv...
Đang xử lý CSC.csv...
Đang xử lý CTB.csv...
Đang xử lý CTC.csv...
Đang xử lý CTP.csv...
Đang xử lý CTT.csv...
Đang xử lý CVN.csv...
Đang xử lý CX8.csv...
Đang xử lý D11.csv...
Đang xử lý DAD.csv...
Đang xử lý DAE.csv...
Đang xử lý DC2.csv...
Đang xử lý DDG.csv...
Đang xử lý DHP.csv...
Đang xử lý DHT.csv...
Đang xử lý DIH.csv...
Đang xử lý DL1.csv...
Đang xử lý DNC.csv...
Đang xử lý DNP.csv...
Đang xử lý DP3.csv...
Đang xử lý

C:\Users\Admin\AppData\Roaming\Python\Python39\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Admin\AppData\Roaming\Python\Python39\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Admin\AppData\Roaming\Python\Python39\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Đang xử lý LDP.csv...
Đang xử lý LHC.csv...
Đang xử lý LIG.csv...
Đang xử lý MAC.csv...
Đang xử lý MAS.csv...
Đang xử lý MBG.csv...
Đang xử lý MBS.csv...
Đang xử lý MCC.csv...
Đang xử lý MCF.csv...
Đang xử lý MCO.csv...
Đang xử lý MDC.csv...
Đang xử lý MED.csv...
Đang xử lý MEL.csv...
Đang xử lý MKV.csv...
Đang xử lý MST.csv...
Đang xử lý MVB.csv...
Đang xử lý NAG.csv...
Đang xử lý NAP.csv...
Đang xử lý NBC.csv...
Đang xử lý NBP.csv...
Đang xử lý NBW.csv...
Đang xử lý NDN.csv...
Đang xử lý NDX.csv...
Đang xử lý NET.csv...
Đang xử lý NFC.csv...
Đang xử lý NHC.csv...
Đang xử lý NRC.csv...
Đang xử lý NSH.csv...
Đang xử lý NST.csv...
Đang xử lý NTH.csv...
Đang xử lý NTP.csv...
Đang xử lý NVB.csv...
Đang xử lý OCH.csv...
Đang xử lý ONE.csv...
Đang xử lý PBP.csv...
Đang xử lý PCE.csv...
Đang xử lý PCG.csv...
Đang xử lý PCH.csv...
Đang xử lý PCT.csv...
Đang xử lý PDB.csv...
Đang xử lý PEN.csv...
Đang xử lý PGN.csv...
Đang xử lý PGS.csv...
Đang xử lý PGT.csv...
Đang xử lý PHN.csv...
Đang xử lý

C:\Users\Admin\AppData\Roaming\Python\Python39\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Admin\AppData\Roaming\Python\Python39\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Admin\AppData\Roaming\Python\Python39\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Đang xử lý QTC.csv...
Đang xử lý RCL.csv...
Đang xử lý S55.csv...
Đang xử lý S99.csv...
Đang xử lý SAF.csv...
Đang xử lý SCG.csv...
Đang xử lý SCI.csv...
Đang xử lý SD5.csv...
Đang xử lý SD9.csv...
Đang xử lý SDA.csv...
Đang xử lý SDC.csv...
Đang xử lý SDG.csv...
Đang xử lý SDN.csv...
Đang xử lý SDU.csv...
Đang xử lý SEB.csv...
Đang xử lý SED.csv...
Đang xử lý SFN.csv...
Đang xử lý SGC.csv...
Đang xử lý SGD.csv...
Đang xử lý SGH.csv...
Đang xử lý SHE.csv...
Đang xử lý SHN.csv...
Đang xử lý SHS.csv...
Đang xử lý SJ1.csv...
Đang xử lý SJE.csv...
Đang xử lý SLS.csv...
Đang xử lý SMN.csv...
Đang xử lý SMT.csv...
Đang xử lý SPC.csv...
Đang xử lý SPI.csv...
Đang xử lý SRA.csv...
Đang xử lý SSM.csv...
Đang xử lý STC.csv...
Đang xử lý STP.csv...
Đang xử lý SVN.csv...
Đang xử lý SZB.csv...
Đang xử lý TA9.csv...
Đang xử lý TBX.csv...


C:\Users\Admin\AppData\Roaming\Python\Python39\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Admin\AppData\Roaming\Python\Python39\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Admin\AppData\Roaming\Python\Python39\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Đang xử lý TDT.csv...
Đang xử lý TET.csv...
Đang xử lý TFC.csv...
Đang xử lý THB.csv...
Đang xử lý THD.csv...
Đang xử lý THS.csv...
Đang xử lý THT.csv...
Đang xử lý TIG.csv...
Đang xử lý TJC.csv...
Đang xử lý TKG.csv...
Đang xử lý TKU.csv...
Đang xử lý TMB.csv...
Đang xử lý TMC.csv...
Đang xử lý TMX.csv...
Đang xử lý TNG.csv...
Đang xử lý TOT.csv...
Đang xử lý TPH.csv...
Đang xử lý TPP.csv...
Đang xử lý TSB.csv...
Đang xử lý TTC.csv...
Đang xử lý TTH.csv...
Đang xử lý TTL.csv...
Đang xử lý TTT.csv...
Đang xử lý TV3.csv...
Đang xử lý TV4.csv...
Đang xử lý TVC.csv...
Đang xử lý TVD.csv...
Đang xử lý TXM.csv...
Đang xử lý UNI.csv...
Đang xử lý V12.csv...
Đang xử lý V21.csv...
Đang xử lý VBC.csv...
Đang xử lý VC1.csv...
Đang xử lý VC2.csv...
Đang xử lý VC3.csv...
Đang xử lý VC6.csv...
Đang xử lý VC7.csv...
Đang xử lý VC9.csv...
Đang xử lý VCC.csv...
Đang xử lý VCM.csv...
Đang xử lý VCS.csv...
Đang xử lý VDL.csv...
Đang xử lý VE1.csv...
Đang xử lý VE3.csv...
Đang xử lý VE4.csv...
Đang xử lý